<a href="https://colab.research.google.com/github/2303A51553/Natural-language-process/blob/main/project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gensim transformers tensorflow nltk scikit-learn rouge-score


import numpy as np
import pandas as pd
import re, string, torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from transformers import BertTokenizer, BertModel
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 31.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b8f3cc02b1fb364267c90b6b939703f500680d4449b6a3dc1903c87dcd134da2
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:

X = pd.read_csv("/content/Legal_Summarisation_100_Final (1).csv")


In [ ]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
X["cleaned_text"] = X["document"].astype(str).apply(preprocess_text)
print("Cleaned Text:\n", X.head())

Cleaned Text:
    id                                           document  \
0   1  The matter concerns constitutional validity of...   
1   2  The matter concerns public interest litigation...   
2   3  The matter concerns tax dispute regarding newl...   
3   4  The matter concerns dispute over custody of a ...   
4   5  The matter concerns public interest litigation...   

                                             summary  \
0  Supreme Court rules on constitutional validity...   
1  Court directs stronger measures for environmen...   
2  Tax provisions upheld as valid and constitutio...   
3  Custody of child decided considering welfare p...   
4  Court directs stronger measures for environmen...   

                                        cleaned_text  
0  matter concern constitutional validity law pet...  
1  matter concern public interest litigation envi...  
2  matter concern tax dispute regarding newly ena...  
3  matter concern dispute custody minor child pet...  
4  matter co

In [ ]:
tfidf = TfidfVectorizer(max_features=10)
tfidf_vectors = tfidf.fit_transform(X["cleaned_text"]).toarray()

print("TF-IDF Embeddings Shape:", tfidf_vectors.shape)
print(pd.DataFrame(tfidf_vectors, columns=tfidf.get_feature_names_out()))


TF-IDF Embeddings Shape: (100, 10)
          14        21    action   article     court  decision   defends  \
0   0.218218  0.218218  0.218218  0.218218  0.654654  0.218218  0.218218   
1   0.204124  0.204124  0.204124  0.204124  0.612372  0.204124  0.204124   
2   0.235702  0.235702  0.235702  0.235702  0.707107  0.235702  0.235702   
3   0.235702  0.235702  0.235702  0.235702  0.707107  0.235702  0.235702   
4   0.204124  0.204124  0.204124  0.204124  0.612372  0.204124  0.204124   
..       ...       ...       ...       ...       ...       ...       ...   
95  0.235702  0.235702  0.235702  0.235702  0.707107  0.235702  0.235702   
96  0.235702  0.235702  0.235702  0.235702  0.707107  0.235702  0.235702   
97  0.235702  0.235702  0.235702  0.235702  0.707107  0.235702  0.235702   
98  0.235702  0.235702  0.235702  0.235702  0.707107  0.235702  0.235702   
99  0.204124  0.204124  0.204124  0.204124  0.612372  0.204124  0.204124   

    interest    public  validity  
0   0.218218  0.2

In [ ]:
tokenized_X = [sentence.split() for sentence in X["cleaned_text"]]

w2v_model = Word2Vec(sentences=tokenized_X, vector_size=100, window=5, min_count=1, workers=4)
word_index = {word: idx + 1 for idx, word in enumerate(w2v_model.wv.index_to_key)}

sequences = [[word_index[word] for word in sent if word in word_index] for sent in tokenized_X]
padded_seq = pad_sequences(sequences, padding='post')

print("Word2Vec Padded Sequences:\n", padded_seq)


Word2Vec Padded Sequences:
 [[24 23 59 ...  0  0  0]
 [24 23  3 ...  5  0  0]
 [24 23 57 ...  6  5  0]
 ...
 [24 23 63 ...  0  0  0]
 [24 23 26 ...  0  0  0]
 [24 23  3 ...  5  0  0]]


In [ ]:
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=100, input_length=padded_seq.shape[1]),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy')

print("LSTM Model Summary:")
model.summary()


🔹 LSTM Model Summary:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=20)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

bert_embeddings = np.vstack([get_bert_embedding(t) for t in X["cleaned_text"]])
print("BERT Embeddings Shape:", bert_embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT Embeddings Shape: (100, 768)


In [ ]:
# BLEU Example
reference = [['the', 'law', 'protects', 'rights', 'citizens']]
candidate = ['law', 'protects', 'citizens']
bleu_score = sentence_bleu(reference, candidate)
print(" BLEU Score:", bleu_score)

# ROUGE Example
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score("law protects rights citizens", "legal systems ensure justice for all")
print(" ROUGE Scores:", scores)


 BLEU Score: 6.439988060393651e-155
 ROUGE Scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [23]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace your_notebook.ipynb

[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 